Imports

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
builder = tfds.builder("cats_vs_dogs")
builder.download_and_prepare()

Dl Completed...:   0%|          | 0/1 [05:02<?, ? url/s]

In [ ]:

train_data = builder.as_dataset(split="train")

for example in train_data.take(5):  # Print the first 5 examples
    image = example["image"]
    label = example["label"]
    filename = example["image/filename"]
    print(f"Label: {label.numpy()}, Filename: {filename.numpy()}")

In [ ]:
# Prepare the data for training and testing
def preprocess_data(data):
    def _preprocess(example):
        image = tf.image.resize(example["image"], (224, 224))
        image = tf.image.convert_image_dtype(image, tf.float32)
        label = tf.one_hot(example["label"], depth=2)
        return image, label

    data = data.map(_preprocess)
    data = data.batch(32)
    data = data.prefetch(tf.data.experimental.AUTOTUNE)
    return data

In [ ]:
train_size = int(0.8 * builder.info.splits["train"].num_examples)
test_size = builder.info.splits["train"].num_examples - train_size

train_data = train_data.take(train_size)
test_data = train_data.skip(train_size).take(test_size)

In [ ]:
train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5
TRAIN_DIR = train_data

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(IMG_SIZE,IMG_SIZE,3),
                   pooling='avg',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))

In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # ResNet50 preprocessing
    validation_split=0.2,  # Use 20% of the data for validation
    # Add more data augmentation settings here if desired
)

In [ ]:
train_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',  # This fetches the training subset
)

In [ ]:
# Validation data generator
val_gen = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',  # This fetches the validation subset
)

In [ ]:
history = resnet_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS
)

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()